In [1]:
from numpy import array
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense


In [2]:
from src.functions_torch import TransformLag
from src.feature import TimeConfig

In [3]:
timec= TimeConfig()
df= timec.process_chain(path='../Data/apple.csv',columns=['Close/Last'])

In [4]:
df= df.loc['2023':'2024'].copy()

In [7]:
timef= TransformLag()
data= timef.lag_transform(df,10)

In [8]:
data

,Value,Value (t-1),Value (t-2),Value (t-3),Value (t-4),Value (t-5),Value (t-6),Value (t-7),Value (t-8),Value (t-9),Value (t-10)
Date,,,,,,,,,,,
2023-02-01,145.43,144.29,143.00,145.93,143.96,141.86,142.53,141.11,137.87,135.27,135.21
2023-02-02,150.82,145.43,144.29,143.00,145.93,143.96,141.86,142.53,141.11,137.87,135.27
2023-02-03,154.50,150.82,145.43,144.29,143.00,145.93,143.96,141.86,142.53,141.11,137.87
2023-02-04,154.50,150.82,145.43,144.29,143.00,145.93,143.96,141.86,142.53,141.11,137.87
2023-02-05,154.50,150.82,145.43,144.29,143.00,145.93,143.96,141.86,142.53,141.11,137.87
...,...,...,...,...,...,...,...,...,...,...,...
2024-05-25,189.98,186.88,190.90,192.35,191.04,189.87,189.84,189.72,187.43,186.28,183.05
2024-05-26,189.98,186.88,190.90,192.35,191.04,189.87,189.84,189.72,187.43,186.28,183.05
2024-05-27,189.98,186.88,190.90,192.35,191.04,189.87,189.84,189.72,187.43,186.28,183.05


In [9]:
data= data.to_numpy()
data

array([[145.43, 144.29, 143.  , ..., 137.87, 135.27, 135.21],
       [150.82, 145.43, 144.29, ..., 141.11, 137.87, 135.27],
       [154.5 , 150.82, 145.43, ..., 142.53, 141.11, 137.87],
       ...,
       [189.98, 186.88, 190.9 , ..., 187.43, 186.28, 183.05],
       [189.99, 189.98, 186.88, ..., 189.72, 187.43, 186.28],
       [190.29, 189.99, 189.98, ..., 189.84, 189.72, 187.43]])

- This is the process from Jason Brownlee in his book Machine Learning Mastery for a vanilla LSTM in TensorFlow

In [1]:
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		end_ix = i + n_steps
		if end_ix > len(sequence)-1:
			break
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]

n_steps = 3

X, y = split_sequence(raw_seq, n_steps)

n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))

model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=200, verbose=0)

x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)

print(yhat)

/Users/main/Desktop/LSTM_Forecast/time/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


[[102.4279]]


- Now lets work under the same architecture to have a a visualization of the forecast provided with tensorflow